# Detecting spouse mentions in sentences

In this tutorial, we will see how Snorkel can be used for Information Extraction. We will walk through an example text classification task for information extraction, where we use labeling functions involving keywords and distant supervision.
### Classification Task
<img src="imgs/sentence.jpg" width="700px;" onerror="this.onerror=null; this.src='/doks-theme/assets/images/sentence.jpg';" align="center" style="display: block; margin-left: auto; margin-right: auto;">

We want to classify each __candidate__ or pair of people mentioned in a sentence, as being married at some point or not.

In the above example, our candidate represents the possible relation `(Barack Obama, Michelle Obama)`. As readers, we know this mention is true due to external knowledge and the keyword of `wedding` occuring later in the sentence.
We begin with some basic setup and data downloading.


In [1]:
# PARAM is significance level for Our Methods and threshold for Varma methods

#PARAM = 0.5
#N_EPS = 50
#LR = 0.1
#DEPS_NAME = 'Varma_Gold' 

#PARAM = 1.5
#N_EPS = 50
#LR = 0.05
#DEPS_NAME = 'Varma' 

PARAM = 0.01
N_EPS = 100
LR = 0.025
DEPS_NAME = 'NM' # NM = New Monitor (Our Method with Gold Labels)

#PARAM = 0.01
#N_EPS = 100
#LR = 0.025
#DEPS_NAME = 'CDGAM' # CDGAM = Our Method without Gold Labels

#PARAM = -1
#N_EPS = 100
#LR = 0.025
#DEPS_NAME = 'Empty' 

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import pickle
import numpy as np

if os.path.basename(os.getcwd()) == "snorkel-tutorials":
    os.chdir("spouse")

In [3]:
from utils import load_data

((df_dev, Y_dev), df_train, (df_test, Y_test)) = load_data()
print(df_dev.shape, df_train.shape, df_test.shape)

(2811, 7) (22254, 7) (2701, 7)


**Input Data:** `df_dev`, `df_train`, and `df_test` are `Pandas DataFrame` objects, where each row represents a particular __candidate__. For our problem, a candidate consists of a sentence, and two people mentioned in the sentence. The DataFrames contain the fields `sentence`, which refers to the sentence of the candidate, `tokens`, the tokenized form of the sentence, and `person1_word_idx` and `person2_word_idx`, which represent `[start, end]` indices in the tokens at which the first and second person's name appear, respectively.

We also have certain **preprocessed fields**, that we discuss a few cells below.

In [4]:

# Don't truncate text fields in the display
pd.set_option("display.max_colwidth", 0)

df_dev.head()

person1_word_idx person2_word_idx  \
0  (1, 1)           (22, 24)          
1  (1, 1)           (7, 8)            
2  (7, 8)           (22, 24)          
3  (6, 6)           (20, 21)          
4  (2, 2)           (4, 5)            

                                                                                                                                                                                                                         sentence  \
0  The Richards are half-sisters to Kathy Hilton, the mother of socialite Paris Hilton and spouse of luxury hotel magnate Richard Howard Hilton.                                                                                    
1  The Richards are half-sisters to Kathy Hilton, the mother of socialite Paris Hilton and spouse of luxury hotel magnate Richard Howard Hilton.                                                                                    
2  The Richards are half-sisters to Kathy Hilton, the mother of socialite Paris Hilton and spouse of luxury hotel magnate Richard Howard Hilton.                                                                                    
3  Prior to both his guests, Colbert's monologue - parts of which he did sitting down - ripped into Donald Trump and his oft-mocked policy of building a wall at the US-Mexico border and not eating Oreos anymore.                 
4  People reported Williams and Ven Veen tied the knot Saturday at Brush Creek Ranch in Saratoga, Wyoming, in front of about 200 guests.                                                                                            

                                                                                                                                                                                                                                                                  tokens  \
0  [The, Richards, are, half, -, sisters, to, Kathy, Hilton, ,, the, mother, of, socialite, Paris, Hilton, and, spouse, of, luxury, hotel, magnate, Richard, Howard, Hilton, ., ]                                                                                          
1  [The, Richards, are, half, -, sisters, to, Kathy, Hilton, ,, the, mother, of, socialite, Paris, Hilton, and, spouse, of, luxury, hotel, magnate, Richard, Howard, Hilton, ., ]                                                                                          
2  [The, Richards, are, half, -, sisters, to, Kathy, Hilton, ,, the, mother, of, socialite, Paris, Hilton, and, spouse, of, luxury, hotel, magnate, Richard, Howard, Hilton, ., ]                                                                                          
3  [Prior, to, both, his, guests, ,, Colbert, s, monologue, -, parts, of, which, he, did, sitting, down, -, ripped, into, Donald, Trump, and, his, oft, -, mocked, policy, of, building, a, wall, at, the, US, -, Mexico, border, and, not, eating, Oreos, anymore, ., ]   
4  [People, reported, Williams, and, Ven, Veen, tied, the, knot, Saturday, at, Brush, Creek, Ranch, in, Saratoga, ,, Wyoming, ,, in, front, of, about, 200, guests, .]                                                                                                     

       person1_right_tokens         person2_right_tokens  \
0  [are, half, -, sisters]   [., ]                         
1  [are, half, -, sisters]   [,, the, mother, of]          
2  [,, the, mother, of]      [., ]                         
3  [s, monologue, -, parts]  [and, his, oft, -]            
4  [and, Ven, Veen, tied]    [tied, the, knot, Saturday]   

                                                                                                                      between_tokens  
0  [are, half, -, sisters, to, Kathy, Hilton, ,, the, mother, of, socialite, Paris, Hilton, and, spouse, of, luxury, hotel, magnate]  
1  [are, half, -, sisters, to]                                                                                                        
2  [,, the, mothe

Let's look at a candidate in the development set:

In [5]:
from preprocessors import get_person_text

candidate = df_dev.loc[2]
person_names = get_person_text(candidate).person_names

print("Sentence: ", candidate["sentence"])
print("Person 1: ", person_names[0])
print("Person 2: ", person_names[1])

Sentence:  The Richards are half-sisters to Kathy Hilton, the mother of socialite Paris Hilton and spouse of luxury hotel magnate Richard Howard Hilton.   
Person 1:  Kathy Hilton
Person 2:  Richard Howard Hilton


### Preprocessing the Data

In a real application, there is a lot of data preparation, parsing, and database loading that needs to be completed before we generate candidates and dive into writing labeling functions. Here we've pre-generated candidates in a pandas DataFrame object per split (train,dev,test).

### Labeling Function Helpers

When writing labeling functions, there are several functions you will use over and over again. In the case of text relation extraction as with this task, common functions include those for fetching text between mentions of the two people in a candidate, examing word windows around person mentions, and so on. We will wrap these functions as `preprocessors`.

In [6]:
from snorkel.preprocess import preprocessor


@preprocessor()
def get_text_between(cand):
    """
    Returns the text between the two person mentions in the sentence for a candidate
    """
    start = cand.person1_word_idx[1] + 1
    end = cand.person2_word_idx[0]
    cand.text_between = " ".join(cand.tokens[start:end])
    return cand

### Candidate PreProcessors

For the purposes of the tutorial, we have three fields (`between_tokens`, `person1_right_tokens`, `person2_right_tokens`) preprocessed in the data, which can be used when creating labeling functions. We also provide the following set of `preprocessor`s for this task in `preprocessors.py`, along with the fields these populate.
* `get_person_text(cand)`: `person_names`
* `get_person_lastnames(cand)`: `person_lastnames`
* `get_left_tokens(cand)`: `person1_left_tokens`, `person2_left_tokens`

In [7]:
from preprocessors import get_left_tokens, get_person_last_names

POSITIVE = 1
NEGATIVE = 0
ABSTAIN = -1

In [8]:
from snorkel.labeling import labeling_function

# Check for the `spouse` words appearing between the person mentions
spouses = {"spouse", "wife", "husband", "ex-wife", "ex-husband"}


@labeling_function(resources=dict(spouses=spouses))
def lf_husband_wife(x, spouses):
    return POSITIVE if len(spouses.intersection(set(x.between_tokens))) > 0 else ABSTAIN

In [9]:
# Check for the `spouse` words appearing to the left of the person mentions
@labeling_function(resources=dict(spouses=spouses), pre=[get_left_tokens])
def lf_husband_wife_left_window(x, spouses):
    if len(set(spouses).intersection(set(x.person1_left_tokens))) > 0:
        return POSITIVE
    elif len(set(spouses).intersection(set(x.person2_left_tokens))) > 0:
        return POSITIVE
    else:
        return ABSTAIN

In [10]:
# Check for the person mentions having the same last name
@labeling_function(pre=[get_person_last_names])
def lf_same_last_name(x):
    p1_ln, p2_ln = x.person_lastnames

    if p1_ln and p2_ln and p1_ln == p2_ln:
        return POSITIVE
    return ABSTAIN

In [11]:
# Check for the word `married` between person mentions
@labeling_function()
def lf_married(x):
    return POSITIVE if "married" in x.between_tokens else ABSTAIN

In [12]:
# Check for words that refer to `family` relationships between and to the left of the person mentions
family = {
    "father",
    "mother",
    "sister",
    "brother",
    "son",
    "daughter",
    "grandfather",
    "grandmother",
    "uncle",
    "aunt",
    "cousin",
}
family = family.union({f + "-in-law" for f in family})


@labeling_function(resources=dict(family=family))
def lf_familial_relationship(x, family):
    return NEGATIVE if len(family.intersection(set(x.between_tokens))) > 0 else ABSTAIN


@labeling_function(resources=dict(family=family), pre=[get_left_tokens])
def lf_family_left_window(x, family):
    if len(set(family).intersection(set(x.person1_left_tokens))) > 0:
        return NEGATIVE
    elif len(set(family).intersection(set(x.person2_left_tokens))) > 0:
        return NEGATIVE
    else:
        return ABSTAIN

In [13]:
# Check for `other` relationship words between person mentions
other = {"boyfriend", "girlfriend", "boss", "employee", "secretary", "co-worker"}


@labeling_function(resources=dict(other=other))
def lf_other_relationship(x, other):
    return NEGATIVE if len(other.intersection(set(x.between_tokens))) > 0 else ABSTAIN

### Distant Supervision Labeling Functions

In addition to using factories that encode pattern matching heuristics, we can also write labeling functions that _distantly supervise_ data points. Here, we'll load in a list of known spouse pairs and check to see if the pair of persons in a candidate matches one of these.

[**DBpedia**](http://wiki.dbpedia.org/): Our database of known spouses comes from DBpedia, which is a community-driven resource similar to Wikipedia but for curating structured data. We'll use a preprocessed snapshot as our knowledge base for all labeling function development.

We can look at some of the example entries from DBPedia and use them in a simple distant supervision labeling function.

Make sure `dbpedia.pkl` is in the `spouse/data` directory.

In [14]:
with open("data/dbpedia.pkl", "rb") as f:
    known_spouses = pickle.load(f)

list(known_spouses)[0:5]

[('Carrie Heffernan', 'Doug Heffernan'),
 ('George Champlin Sibley', 'Mary Easton Sibley'),
 ('Greg Benson', 'Paige Davis'),
 ('Chandrika Kumaratunga', 'Vijaya Kumaranatunga'),
 ('Catherine of Austria', 'Charles I Margrave of Baden-Baden')]

In [15]:
@labeling_function(resources=dict(known_spouses=known_spouses), pre=[get_person_text])
def lf_distant_supervision(x, known_spouses):
    p1, p2 = x.person_names
    if (p1, p2) in known_spouses or (p2, p1) in known_spouses:
        return POSITIVE
    else:
        return ABSTAIN

In [16]:
from preprocessors import last_name

# Last name pairs for known spouses
last_names = set(
    [
        (last_name(x), last_name(y))
        for x, y in known_spouses
        if last_name(x) and last_name(y)
    ]
)


@labeling_function(resources=dict(last_names=last_names), pre=[get_person_last_names])
def lf_distant_supervision_last_names(x, last_names):
    p1_ln, p2_ln = x.person_lastnames

    return (
        POSITIVE
        if (p1_ln != p2_ln)
        and ((p1_ln, p2_ln) in last_names or (p2_ln, p1_ln) in last_names)
        else ABSTAIN
    )

#### Apply Labeling Functions to the Data
We create a list of labeling functions and apply them to the data

In [17]:
from snorkel.labeling import PandasLFApplier

lfs = [
    lf_husband_wife,
    lf_husband_wife_left_window,
    lf_same_last_name,
    lf_married,
    lf_familial_relationship,
    lf_family_left_window,
    lf_other_relationship,
    lf_distant_supervision,
    lf_distant_supervision_last_names,
]
applier = PandasLFApplier(lfs)

In [18]:
from snorkel.labeling import LFAnalysis

L_dev = applier.apply(df_dev)
L_train = applier.apply(df_train)

100%|██████████| 22254/22254 [00:51<00:00, 433.52it/s]


In [19]:
LFAnalysis(L_dev, lfs).lf_summary(Y_dev)

j Polarity  Coverage  Overlaps  Conflicts  \
lf_husband_wife                    0  [1]      0.089648  0.036642  0.017432    
lf_husband_wife_left_window        1  [1]      0.025258  0.021345  0.003557    
lf_same_last_name                  2  [1]      0.040555  0.016009  0.008538    
lf_married                         3  [1]      0.019210  0.006759  0.002490    
lf_familial_relationship           4  [0]      0.115617  0.051939  0.026325    
lf_family_left_window              5  [0]      0.041266  0.033440  0.007826    
lf_other_relationship              6  [0]      0.013874  0.002846  0.002846    
lf_distant_supervision             7  [1]      0.001067  0.001067  0.000000    
lf_distant_supervision_last_names  8  [1]      0.001067  0.000711  0.000356    

                                   Correct  Incorrect  Emp. Acc.  
lf_husband_wife                    93       159        0.369048   
lf_husband_wife_left_window        30       41         0.422535   
lf_same_last_name                  19       95         0.166667   
lf_married                         22       32         0.407407   
lf_familial_relationship           310      15         0.953846   
lf_family_left_window              114      2          0.982759   
lf_other_relationship              33       6          0.846154   
lf_distant_supervision             2        1          0.666667   
lf_distant_supervision_last_names  0        3          0.000000

## Part 3b: Finding dependencies; Training Snorkel label_model

In [20]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
from Our_Monitors.CD_Monitor import CDM, Informed_LabelModel
from Our_Monitors.CDGA_Monitor import CDGAM
from Our_Monitors.New_Monitor import NM
from Our_Monitors.utils import ModVarma_InCov

from dependency_model.varma_deps_functions import get_varma_edges, get_varma_with_gold_edges

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
def overall_deps_fn(deps_name, param):
    if deps_name == 'CDM':
        deps = CDM(L_dev, Y_dev, k=2, sig=param, policy = 'old', verbose=False, return_more_info = False)
    elif deps_name == 'CDGAM':
        deps = CDGAM(L_dev, k=2, sig=param, policy = 'old', verbose = False, return_more_info = False)
    elif deps_name == 'NM':
        deps = NM(L_dev, Y_dev, k=2, sig=param, policy = 'old', verbose=False, return_more_info = False)
    elif deps_name == 'NM_NP':
        deps = NM(L_dev, Y_dev, k=2, sig=param, policy = 'new', verbose=False, return_more_info = False)
    elif deps_name == 'Mod_Varma':
        deps = ModVarma_InCov(L_dev, Y_dev, thresh=param)
    elif deps_name == 'Varma':
        deps = get_varma_edges(L_dev, thresh=param)
    elif deps_name == 'Varma_Gold':
        deps = get_varma_with_gold_edges(L_dev, Y_dev, thresh=param)
    elif deps_name == 'Empty':
        deps = []
    return deps

deps = overall_deps_fn(DEPS_NAME, PARAM)
print(deps)

[(0, 1), (4, 5), (7, 8)]


In [22]:
# init informed model
label_model = Informed_LabelModel(edges = deps, cardinality=2, verbose=True)
# train informed model
label_model.fit(L_train, Y_dev, n_epochs=N_EPS, log_freq=N_EPS/10, seed=12345, lr=LR)

/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype

### Label Model Metrics
Since our dataset is highly unbalanced (91% of the labels are negative), even a trivial baseline that always outputs negative can get a high accuracy. So we evaluate the label model using the F1 score and ROC-AUC rather than accuracy.

In [23]:
from snorkel.analysis import metric_score
from snorkel.utils import probs_to_preds

probs_dev = label_model.predict_proba(L_dev)
preds_dev = probs_to_preds(probs_dev)
print("On valid set...")
print(f"Label model f1 score: {metric_score(Y_dev, preds_dev, probs=probs_dev, metric='f1')}")
print(f"Label model roc-auc: {metric_score(Y_dev, preds_dev, probs=probs_dev, metric='roc_auc')}")

L_test = applier.apply(df_test)
probs_test = label_model.predict_proba(L_test)
preds_test = probs_to_preds(probs_test)
print("On test set...")
print(f"Label model f1 score: {metric_score(Y_test, preds_test, probs=probs_test, metric='f1')}")
print(f"Label model roc-auc: {metric_score(Y_test, preds_test, probs=probs_test, metric='roc_auc')}")

  0%|          | 0/2701 [00:00<?, ?it/s]

On valid set...
Label model f1 score: 0.42654028436018954
Label model roc-auc: 0.7777003554288239


100%|██████████| 2701/2701 [00:07<00:00, 342.53it/s]

On test set...
Label model f1 score: 0.4990176817288801
Label model roc-auc: 0.7953450435438043


### Part 4: Training our End Extraction Model

In this final section of the tutorial, we'll use our noisy training labels to train our end machine learning model. We start by filtering out training data points which did not recieve a label from any LF, as these data points contain no signal.


In [24]:
from snorkel.labeling import filter_unlabeled_dataframe

probs_train = label_model.predict_proba(L_train)
df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(X=df_train, y=probs_train, L=L_train)

Next, we train a simple [LSTM](https://en.wikipedia.org/wiki/Long_short-term_memory) network for classifying candidates. `tf_model` contains functions for processing features and building the keras model for training and evaluation.

In [25]:
from tf_model import get_model, get_feature_arrays
from utils import get_n_epochs

X_train = get_feature_arrays(df_train_filtered)
model = get_model()
batch_size = 64
model.fit(X_train, probs_train_filtered, batch_size=batch_size, epochs=get_n_epochs())

/home/kaustubh-anaconda/anaconda3/envs/snorkel_env_clone/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kaustubh-anaconda/anaconda3/envs/snorkel_env_clone/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kaustubh-anaconda/anaconda3/envs/snorkel_env_clone/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

Epoch 1/30
5734/5734 [==============================] - 12s 2ms/sample - loss: 0.6581
Epoch 2/30
5734/5734 [==============================] - 9s 2ms/sample - loss: 0.6557
Epoch 3/30
5734/5734 [==============================] - 9s 2ms/sample - loss: 0.6516
Epoch 4/30
5734/5734 [==============================] - 10s 2ms/sample - loss: 0.6272
Epoch 5/30
5734/5734 [==============================] - 9s 2ms/sample - loss: 0.5914
Epoch 6/30
5734/5734 [==============================] - 9s 2ms/sample - loss: 0.5582
Epoch 7/30
5734/5734 [==============================] - 9s 2ms/sample - loss: 0.5324
Epoch 8/30
5734/5734 [==============================] - 10s 2ms/sample - loss: 0.5173
Epoch 9/30
5734/5734 [==============================] - 9s 2ms/sample - loss: 0.5010
Epoch 10/30
5734/5734 [==============================] - 9s 2ms/sample - loss: 0.4907
Epoch 11/30
5734/5734 [==============================] - 10s 2ms/sample - loss: 0.4822
Epoch 12/30
5734/5734 [==============================] - 9s

Finally, we evaluate the trained model by measuring its F1 score and ROC_AUC.

In [26]:
X_test = get_feature_arrays(df_test)
probs_test = model.predict(X_test)
preds_test = probs_to_preds(probs_test)
print(f"Test F1 when trained with soft labels: {metric_score(Y_test, preds=preds_test, metric='f1')}")
print(f"Test ROC-AUC when trained with soft labels: {metric_score(Y_test, probs=probs_test, metric='roc_auc')}")
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(Y_test, preds_test, pos_label=1, average = 'binary'))

Test F1 when trained with soft labels: 0.30062630480167013
Test ROC-AUC when trained with soft labels: 0.7761696970592691
(0.1945945945945946, 0.6605504587155964, 0.30062630480167013, None)
